## SF-crime Kaggle competition

#### This is an attempt to build a standard tabular data model using Random Forest on Kaggle data

In [1]:
# Let's start by some standard code
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.imports import *
from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestClassifier
from IPython.display import display

from sklearn import metrics
from graphviz import *

In [3]:
PATH = "data/Kaggle/competitions/sf-crime/"

In [4]:
!ls {PATH}

sampleSubmission.csv test.csv             train.csv


### 1. Let's load and explore the data

In [5]:
df_raw=pd.read_csv(f'{PATH}train.csv')

In [6]:
df_raw.columns

Index(['Dates', 'Category', 'Descript', 'DayOfWeek', 'PdDistrict',
       'Resolution', 'Address', 'X', 'Y'],
      dtype='object')

In [7]:
df_raw=pd.read_csv(f'{PATH}train.csv',parse_dates=['Dates'])

In [8]:
df_raw.shape

(878049, 9)

In [9]:
display(df_raw.head())

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414
3,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873
4,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541


In [10]:
display(df_raw.isnull().sum().sort_index()/len(df_raw))

Address       0.0
Category      0.0
Dates         0.0
DayOfWeek     0.0
Descript      0.0
PdDistrict    0.0
Resolution    0.0
X             0.0
Y             0.0
dtype: float64

### What we see in the data
1. One date field. So the date formatting and feature extraction is required.
2. A few categorical variables. We will have to convert these into numbers.
3. No missing values.

In [91]:
df_raw['Category'].unique()

[WARRANTS, OTHER OFFENSES, LARCENY/THEFT, VEHICLE THEFT, VANDALISM, ..., GAMBLING, BAD CHECKS, TREA, RECOVERED VEHICLE, PORNOGRAPHY/OBSCENE MAT]
Length: 39
Categories (39, object): [ARSON < ASSAULT < BAD CHECKS < BRIBERY ... VANDALISM < VEHICLE THEFT < WARRANTS < WEAPON LAWS]

In [12]:
df_raw['PdDistrict'].unique()

array(['NORTHERN', 'PARK', 'INGLESIDE', 'BAYVIEW', 'RICHMOND', 'CENTRAL', 'TARAVAL', 'TENDERLOIN', 'MISSION',
       'SOUTHERN'], dtype=object)

We seem to have some missing values. Let's explore summary stats.

### 2. Format the data

In [13]:
df_raw.dtypes

Dates         datetime64[ns]
Category              object
Descript              object
DayOfWeek             object
PdDistrict            object
Resolution            object
Address               object
X                    float64
Y                    float64
dtype: object

We would not see the codes in the actual dataframe but we can check using the following code.

### We need extract feature from the Date-Time field

In [14]:
# You should always consider this feature extraction step when working with date-time.
add_datepart(df_raw,fldname='Dates',time=True)
df_raw.head()

,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y,DatesYear,DatesMonth,...,DatesIs_month_end,DatesIs_month_start,DatesIs_quarter_end,DatesIs_quarter_start,DatesIs_year_end,DatesIs_year_start,DatesHour,DatesMinute,DatesSecond,DatesElapsed
0,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599,2015,5,...,False,False,False,False,False,False,23,53,0,1431561180
1,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599,2015,5,...,False,False,False,False,False,False,23,53,0,1431561180
2,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414,2015,5,...,False,False,False,False,False,False,23,33,0,1431559980
3,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873,2015,5,...,False,False,False,False,False,False,23,30,0,1431559800
4,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541,2015,5,...,False,False,False,False,False,False,23,30,0,1431559800


### We need to tell the program which variables should be treated categorically

In [15]:
df_raw.columns

Index(['Category', 'Descript', 'DayOfWeek', 'PdDistrict', 'Resolution',
       'Address', 'X', 'Y', 'DatesYear', 'DatesMonth', 'DatesWeek', 'DatesDay',
       'DatesDayofweek', 'DatesDayofyear', 'DatesIs_month_end',
       'DatesIs_month_start', 'DatesIs_quarter_end', 'DatesIs_quarter_start',
       'DatesIs_year_end', 'DatesIs_year_start', 'DatesHour', 'DatesMinute',
       'DatesSecond', 'DatesElapsed'],
      dtype='object')

In [16]:
df_raw[['DayOfWeek','DatesDayofweek']].head()

,DayOfWeek,DatesDayofweek
0,Wednesday,2
1,Wednesday,2
2,Wednesday,2
3,Wednesday,2
4,Wednesday,2


Let's delete the duplicate column.

In [17]:
df_raw.drop('DayOfWeek',axis=1,inplace=True)

Also, the "Descript" and "Resolution" columns are part of only training dataset and not the target variable. So let's drop that as well.

In [18]:
df_raw.drop(['Descript','Resolution'],axis=1,inplace=True)

In [19]:
df_raw.dtypes

Category                  object
PdDistrict                object
Address                   object
X                        float64
Y                        float64
DatesYear                  int64
DatesMonth                 int64
DatesWeek                  int64
DatesDay                   int64
DatesDayofweek             int64
DatesDayofyear             int64
DatesIs_month_end           bool
DatesIs_month_start         bool
DatesIs_quarter_end         bool
DatesIs_quarter_start       bool
DatesIs_year_end            bool
DatesIs_year_start          bool
DatesHour                  int64
DatesMinute                int64
DatesSecond                int64
DatesElapsed               int64
dtype: object

Perhaps no explicit datatype conversions are required into a category type.

In [20]:
df_raw.PdDistrict.cat

AttributeError: Can only use .cat accessor with a 'category' dtype

In [21]:
train_cats(df_raw)

In [22]:
df_raw.PdDistrict.cat.codes.head()
# Before calling "train_cats" we got error for this, but now as we can see the codes are ready.

0    4
1    4
2    4
3    4
4    5
dtype: int8

In [23]:
# Let's take a backup of original dataframe
#df = df_raw

In [24]:
# No explicit conversion is required
# for v in cat_vars: df[v] = df[v].astype('category')

In [25]:
# Now although we have generated coded for Categorical variables we havn't used them in the orginal dataframe
df_raw.head()

,Category,PdDistrict,Address,X,Y,DatesYear,DatesMonth,DatesWeek,DatesDay,DatesDayofweek,...,DatesIs_month_end,DatesIs_month_start,DatesIs_quarter_end,DatesIs_quarter_start,DatesIs_year_end,DatesIs_year_start,DatesHour,DatesMinute,DatesSecond,DatesElapsed
0,WARRANTS,NORTHERN,OAK ST / LAGUNA ST,-122.425892,37.774599,2015,5,20,13,2,...,False,False,False,False,False,False,23,53,0,1431561180
1,OTHER OFFENSES,NORTHERN,OAK ST / LAGUNA ST,-122.425892,37.774599,2015,5,20,13,2,...,False,False,False,False,False,False,23,53,0,1431561180
2,OTHER OFFENSES,NORTHERN,VANNESS AV / GREENWICH ST,-122.424363,37.800414,2015,5,20,13,2,...,False,False,False,False,False,False,23,33,0,1431559980
3,LARCENY/THEFT,NORTHERN,1500 Block of LOMBARD ST,-122.426995,37.800873,2015,5,20,13,2,...,False,False,False,False,False,False,23,30,0,1431559800
4,LARCENY/THEFT,PARK,100 Block of BRODERICK ST,-122.438738,37.771541,2015,5,20,13,2,...,False,False,False,False,False,False,23,30,0,1431559800


In [26]:
# We can add fix_missing as one of the parameter. 
# It works only on numeric, as pandas automatically takes care of categorical variable missing values 
# by setting them to -1.
df_x, y, nas = proc_df(df_raw,'Category')

In [27]:
# Now it should all be numeric or boolean
df_x.head()

,PdDistrict,Address,X,Y,DatesYear,DatesMonth,DatesWeek,DatesDay,DatesDayofweek,DatesDayofyear,DatesIs_month_end,DatesIs_month_start,DatesIs_quarter_end,DatesIs_quarter_start,DatesIs_year_end,DatesIs_year_start,DatesHour,DatesMinute,DatesSecond,DatesElapsed
0,5,19791,-122.425892,37.774599,2015,5,20,13,2,133,False,False,False,False,False,False,23,53,0,1431561180
1,5,19791,-122.425892,37.774599,2015,5,20,13,2,133,False,False,False,False,False,False,23,53,0,1431561180
2,5,22698,-122.424363,37.800414,2015,5,20,13,2,133,False,False,False,False,False,False,23,33,0,1431559980
3,5,4267,-122.426995,37.800873,2015,5,20,13,2,133,False,False,False,False,False,False,23,30,0,1431559800
4,6,1844,-122.438738,37.771541,2015,5,20,13,2,133,False,False,False,False,False,False,23,30,0,1431559800


In [28]:
y[:10]

array([37, 21, 21, 16, 16, 16, 36, 36, 16, 16], dtype=int8)

In [29]:
np.unique(y)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24,
       25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38], dtype=int8)

### 3. Build the model

In [30]:
# n_jobs=-1 helps us to run the jobs in parallel (one per CPU core)
m = RandomForestClassifier(n_jobs=-1)
# Build a forest of trees from the training set (X, y).
m.fit(df_x, y)
# The score() Returns the mean accuracy on the given test data and labels.
m.score(df_x,y)

0.8495129542884281

This model we have is purely on training data, so the high score is not necessarily a good sign.

### Let's load the test data and process it.

In [31]:
df_raw_test = pd.read_csv(f'{PATH}test.csv',date_parser='Dates')

In [32]:
df_raw_test.head()

,Id,Dates,DayOfWeek,PdDistrict,Address,X,Y
0,0,2015-05-10 23:59:00,Sunday,BAYVIEW,2000 Block of THOMAS AV,-122.399588,37.735051
1,1,2015-05-10 23:51:00,Sunday,BAYVIEW,3RD ST / REVERE AV,-122.391523,37.732432
2,2,2015-05-10 23:50:00,Sunday,NORTHERN,2000 Block of GOUGH ST,-122.426002,37.792212
3,3,2015-05-10 23:45:00,Sunday,INGLESIDE,4700 Block of MISSION ST,-122.437394,37.721412
4,4,2015-05-10 23:45:00,Sunday,INGLESIDE,4700 Block of MISSION ST,-122.437394,37.721412


In [33]:
add_datepart(df=df_raw_test,fldname='Dates',time=True)

In [34]:
df_raw_test.columns

Index(['Id', 'DayOfWeek', 'PdDistrict', 'Address', 'X', 'Y', 'DatesYear',
       'DatesMonth', 'DatesWeek', 'DatesDay', 'DatesDayofweek',
       'DatesDayofyear', 'DatesIs_month_end', 'DatesIs_month_start',
       'DatesIs_quarter_end', 'DatesIs_quarter_start', 'DatesIs_year_end',
       'DatesIs_year_start', 'DatesHour', 'DatesMinute', 'DatesSecond',
       'DatesElapsed'],
      dtype='object')

In [35]:
# training dataset column
df_raw.columns

Index(['Category', 'PdDistrict', 'Address', 'X', 'Y', 'DatesYear',
       'DatesMonth', 'DatesWeek', 'DatesDay', 'DatesDayofweek',
       'DatesDayofyear', 'DatesIs_month_end', 'DatesIs_month_start',
       'DatesIs_quarter_end', 'DatesIs_quarter_start', 'DatesIs_year_end',
       'DatesIs_year_start', 'DatesHour', 'DatesMinute', 'DatesSecond',
       'DatesElapsed'],
      dtype='object')

In [36]:
# Let's drop the duplicate column - DayOfWeek and the extra 'Id' column.
df_raw_test.drop(['DayOfWeek','Id'],axis=1,inplace=True)

In [37]:
df_raw_test.head()

,PdDistrict,Address,X,Y,DatesYear,DatesMonth,DatesWeek,DatesDay,DatesDayofweek,DatesDayofyear,DatesIs_month_end,DatesIs_month_start,DatesIs_quarter_end,DatesIs_quarter_start,DatesIs_year_end,DatesIs_year_start,DatesHour,DatesMinute,DatesSecond,DatesElapsed
0,BAYVIEW,2000 Block of THOMAS AV,-122.399588,37.735051,2015,5,19,10,6,130,False,False,False,False,False,False,23,59,0,1431302340
1,BAYVIEW,3RD ST / REVERE AV,-122.391523,37.732432,2015,5,19,10,6,130,False,False,False,False,False,False,23,51,0,1431301860
2,NORTHERN,2000 Block of GOUGH ST,-122.426002,37.792212,2015,5,19,10,6,130,False,False,False,False,False,False,23,50,0,1431301800
3,INGLESIDE,4700 Block of MISSION ST,-122.437394,37.721412,2015,5,19,10,6,130,False,False,False,False,False,False,23,45,0,1431301500
4,INGLESIDE,4700 Block of MISSION ST,-122.437394,37.721412,2015,5,19,10,6,130,False,False,False,False,False,False,23,45,0,1431301500


In [38]:
# Now, convert text into numbers
# Changes any columns of strings in df into categorical variables using trn as a template for the category codes.
apply_cats(df=df_raw_test,trn=df_raw)

In [39]:
# Let's confirm if it worked
df_raw_test.PdDistrict.cat.codes.head()

0    0
1    0
2    4
3    2
4    2
dtype: int8

In [40]:
df_raw_test.head()

,PdDistrict,Address,X,Y,DatesYear,DatesMonth,DatesWeek,DatesDay,DatesDayofweek,DatesDayofyear,DatesIs_month_end,DatesIs_month_start,DatesIs_quarter_end,DatesIs_quarter_start,DatesIs_year_end,DatesIs_year_start,DatesHour,DatesMinute,DatesSecond,DatesElapsed
0,BAYVIEW,2000 Block of THOMAS AV,-122.399588,37.735051,2015,5,19,10,6,130,False,False,False,False,False,False,23,59,0,1431302340
1,BAYVIEW,3RD ST / REVERE AV,-122.391523,37.732432,2015,5,19,10,6,130,False,False,False,False,False,False,23,51,0,1431301860
2,NORTHERN,2000 Block of GOUGH ST,-122.426002,37.792212,2015,5,19,10,6,130,False,False,False,False,False,False,23,50,0,1431301800
3,INGLESIDE,4700 Block of MISSION ST,-122.437394,37.721412,2015,5,19,10,6,130,False,False,False,False,False,False,23,45,0,1431301500
4,INGLESIDE,4700 Block of MISSION ST,-122.437394,37.721412,2015,5,19,10,6,130,False,False,False,False,False,False,23,45,0,1431301500


In [41]:
df_raw_test.dtypes

PdDistrict               category
Address                  category
X                         float64
Y                         float64
DatesYear                   int64
DatesMonth                  int64
DatesWeek                   int64
DatesDay                    int64
DatesDayofweek              int64
DatesDayofyear              int64
DatesIs_month_end            bool
DatesIs_month_start          bool
DatesIs_quarter_end          bool
DatesIs_quarter_start        bool
DatesIs_year_end             bool
DatesIs_year_start           bool
DatesHour                   int64
DatesMinute                 int64
DatesSecond                 int64
DatesElapsed                int64
dtype: object

In [42]:
df_test_x = proc_df(df_raw_test)

In [43]:
df_test_x[0].head()

,PdDistrict,Address,X,Y,DatesYear,DatesMonth,DatesWeek,DatesDay,DatesDayofweek,DatesDayofyear,DatesIs_month_end,DatesIs_month_start,DatesIs_quarter_end,DatesIs_quarter_start,DatesIs_year_end,DatesIs_year_start,DatesHour,DatesMinute,DatesSecond,DatesElapsed
0,1,6421,-122.399588,37.735051,2015,5,19,10,6,130,False,False,False,False,False,False,23,59,0,1431302340
1,1,9761,-122.391523,37.732432,2015,5,19,10,6,130,False,False,False,False,False,False,23,51,0,1431301860
2,5,6350,-122.426002,37.792212,2015,5,19,10,6,130,False,False,False,False,False,False,23,50,0,1431301800
3,3,10657,-122.437394,37.721412,2015,5,19,10,6,130,False,False,False,False,False,False,23,45,0,1431301500
4,3,10657,-122.437394,37.721412,2015,5,19,10,6,130,False,False,False,False,False,False,23,45,0,1431301500


In [44]:
type(df_test_x[0])

pandas.core.frame.DataFrame

Now let's try the model on this test data

In [45]:
df_test_x[0].shape

(884262, 20)

In [46]:
df_x.columns

Index(['PdDistrict', 'Address', 'X', 'Y', 'DatesYear', 'DatesMonth',
       'DatesWeek', 'DatesDay', 'DatesDayofweek', 'DatesDayofyear',
       'DatesIs_month_end', 'DatesIs_month_start', 'DatesIs_quarter_end',
       'DatesIs_quarter_start', 'DatesIs_year_end', 'DatesIs_year_start',
       'DatesHour', 'DatesMinute', 'DatesSecond', 'DatesElapsed'],
      dtype='object')

In [47]:
df_test_x[0].columns

Index(['PdDistrict', 'Address', 'X', 'Y', 'DatesYear', 'DatesMonth',
       'DatesWeek', 'DatesDay', 'DatesDayofweek', 'DatesDayofyear',
       'DatesIs_month_end', 'DatesIs_month_start', 'DatesIs_quarter_end',
       'DatesIs_quarter_start', 'DatesIs_year_end', 'DatesIs_year_start',
       'DatesHour', 'DatesMinute', 'DatesSecond', 'DatesElapsed'],
      dtype='object')

In [48]:
df_test = df_test_x[0]

In [49]:
df_test.columns

Index(['PdDistrict', 'Address', 'X', 'Y', 'DatesYear', 'DatesMonth',
       'DatesWeek', 'DatesDay', 'DatesDayofweek', 'DatesDayofyear',
       'DatesIs_month_end', 'DatesIs_month_start', 'DatesIs_quarter_end',
       'DatesIs_quarter_start', 'DatesIs_year_end', 'DatesIs_year_start',
       'DatesHour', 'DatesMinute', 'DatesSecond', 'DatesElapsed'],
      dtype='object')

In [50]:
results = m.predict(df_test)

In [58]:
results.shape

(884262,)

In [61]:
results[1:50]

array([21, 16, 21, 21, 16, 21, 16, 32, 16, 16, 20, 20, 16, 16,  7, 16, 21, 16, 16, 16, 20, 20,  1, 16, 36,
       20, 16, 16, 16, 16, 36, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 20, 16, 16, 16, 16, 16, 16, 36],
      dtype=int8)

In [64]:
np.unique(results)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 23, 24, 25,
       26, 27, 28, 29, 30, 31, 32, 34, 35, 36, 37, 38], dtype=int8)

We have the predictions for each of the test set example.

### Pulse-check: Submit the existing predictions to Kaggle and check the accuracy

In [97]:
len(results)

884262

In [131]:
submission_file_accumulator=list()

for i in range(0,len(results)):
    temp=np.zeros(39,dtype=int)
    temp[results[i]]=1
    line=np.insert(temp,0,i)
    submission_file_accumulator.append(line)

In [134]:
len(submission_file_accumulator)

884262

In [135]:
tt = pd.DataFrame(submission_file_accumulator)

In [137]:
tt.shape

(884262, 40)

In [160]:
tt.to_csv('data/Kaggle/competitions/sf-crime/result_v1.csv',index=False)

In [149]:
results[21:40]+1

array([21, 21,  2, 17, 37, 21, 17, 17, 17, 17, 37, 17, 17, 17, 17, 17, 17, 17, 17], dtype=int8)

In [142]:
np.unique(results)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 23, 24, 25,
       26, 27, 28, 29, 30, 31, 32, 34, 35, 36, 37, 38], dtype=int8)

In [159]:
tt.iloc[0:4,0:6]

,0,1,2,3,4,5
0,0,0,0,0,0,0
1,1,0,0,0,0,0
2,2,0,0,0,0,0
3,3,0,0,0,0,0


In [157]:
tt.columns.values[0:5]

array([0, 1, 2, 3, 4])